# Analysis of Agent-Based Simulation

In [2]:
# prepare for Python version 3x features and functions
from __future__ import division, print_function

# import packages into the namespace for this program
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [3]:
# -----------------------------
# Simulation study background
# -----------------------------
# an agent-based simulation was run with NetLogo, a public-domain
# program available from Northwestern University

In [21]:
# added one line of code to the Virus on a Network program:

if ticks == 200:[stop]

NameError: name 'ticks' is not defined

In [12]:
# this line was added to stop the simulation at exactly 200 ticks
# the line was added to the <to go> code block as shown here:
# 
# to go
#   if all? turtles [not infected?]
#     [ stop ]
#   ask turtles
#   [
#      set virus-check-timer virus-check-timer + 1
#      if virus-check-timer >= virus-check-frequency
#        [ set virus-check-timer 0 ]
#   ]
#   if ticks = 200 [stop]
#   spread-virus
#   do-virus-checks
#   tick
# end

# the simulation stops if no nodes/turtles were infected 
# or if the simulation reaches 200 ticks

# To see the results of the simulation at 200 ticks, we route the simulation
# world to a file using the GUI File/Export/Export World  
# this gives an a comma-delimited text file of the status of the network 
# at 200 ticks. Specifically, we enter the following Excel command into 
# cell D1 of the results spreadsheet to compute the proportion of nodes 
# infected:   = COUNTIF(N14:N163, TRUE)/M10

# NetLogo turtle infected status values were given in cells N14 through N163. 
# The detailed results of the simulation runs or trials are shown in the files 
# <trial01.csv> through <trial20.csv> under the directory NetLogo_results

# this particular experiment, has average connectivity or node degree 
# at 3 or 5 and the susceptibility or virus spread chance to 5 or 10 percent. 
# we have a completely crossed 2 x 2 design with 5 replications of each cell
# that is, we run each treatment combination 5 times, 20 independent 
# observations or trials. for each trial, we note the percentage of infected 
# nodes after 200 ticks---this is the response variable
# results are summarized in the comma-delimited file <virus_results.csv>. 

# -----------------------------
# Analysis of Deviance
# -----------------------------

In [22]:
# read in summary results and code the experimental factors
virus = pd.read_csv("virus_results.csv")


In [23]:
# check input DataFrame
print(virus)


    trial  degree  spread  infected
0       1       3       5    0.1133
1       2       3       5    0.1533
2       3       3       5    0.0867
3       4       3       5    0.1400
4       5       3       5    0.1133
5       6       3      10    0.2000
6       7       3      10    0.1067
7       8       3      10    0.0867
8       9       3      10    0.4133
9      10       3      10    0.4267
10     11       5       5    0.3067
11     12       5       5    0.4867
12     13       5       5    0.4867
13     14       5       5    0.4667
14     15       5       5    0.6800
15     16       5      10    0.4733
16     17       5      10    0.5133
17     18       5      10    0.5400
18     19       5      10    0.5933
19     20       5      10    0.5667


In [24]:
Intercept = np.array([1] * len(virus))

In [25]:
# use dictionary object for mapping to 0/1 binary codes
degree_to_binary = {3 : 0, 5 : 1}
Connectivity = np.array(virus['degree'].map(degree_to_binary))

In [26]:
# use dictionary object for mapping to 0/1 binary codes
spread_to_binary = {5 : 0, 10 : 1}
Susceptibility = np.array(virus['spread'].map(spread_to_binary))

In [27]:
Connectivity_Susceptibility = Connectivity * Susceptibility

Design_Matrix = np.array([Intercept, Connectivity, Susceptibility, Connectivity_Susceptibility]).T

print(Design_Matrix)


[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 1 0 0]
 [1 1 0 0]
 [1 1 0 0]
 [1 1 0 0]
 [1 1 0 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]


In [28]:
Market_Share = np.array(virus['infected'])

In [29]:
# generalized linear model for a response variable that is a proportion
glm_binom = sm.GLM(Market_Share, Design_Matrix, family=sm.families.Binomial())
res = glm_binom.fit()
print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   20
Model:                            GLM   Df Residuals:                       16
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -7.9002
Date:                Mon, 24 Jul 2023   Deviance:                      0.94434
Time:                        21:14:49   Pearson chi2:                    0.920
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1275
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9800      1.370     -1.446      0.1